In [ ]:
import os
os.chdir('C:\\Github')

In [ ]:
import pickle
import numpy as np
import csv
import pandas as pd
from collections import Counter

In [ ]:
# table[y][x]
table = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
        [ 0, 1, 0, 0, 0, 0,11, 0, 0, 0, 0,21],\
        [ 0, 2, 0, 0, 0, 0,12, 0, 0, 0, 0,22],\
        [ 0, 3, 0, 0, 0, 0,13, 0, 0, 0, 0,23],\
        [ 0, 4, 0, 0, 0, 0,14, 0, 0, 0, 0,24],\
        [ 0, 5, 0, 0, 0, 0,15, 0, 0, 0, 0,25],\
        [ 0, 6, 0, 0, 0, 0,16, 0, 0, 0, 0,26],\
        [ 0, 7, 0, 0, 0, 0,17, 0, 0, 0, 0,27],\
        [ 0, 8, 0, 0, 0, 0,18, 0, 0, 0, 0,28],\
        [ 0, 9, 0, 0, 0, 0,19, 0, 0, 0, 0,29],\
        [ 0,10, 0, 0, 0, 0,20, 0, 0, 0, 0,30],\
        [ 0,31,32,33,34,35,36,37,38,39,40,41] ], dtype = int)

def class_to_coordinate(a):
    x = np.argwhere(table == a)[0][1]
    y = np.argwhere(table == a)[0][0]
    coordinate = [x,y]
    return coordinate

def uuid_to_b_num(s):
    if s[0:30] == 'e20a39f4-73f5-4bc4-a12f-17d1ad' :
        uuid_num = int(s[-1])
    else :
        uuid_num = 0
    return uuid_num

## 說明：
- 透過當初在每個位置收的 BLE_data.csv，產生wireless fingerprint training set
，一共生成三個檔案：
![](https://i.imgur.com/YBkSogr.png)

- max_rssi.pickle 跟 min_rssi.pickle 分別儲存每支手機的 RSSI 最大/最小值，以便進行 MIN-MAX Normalization

## 參數:
- 無


In [ ]:
wireless_fingerprint = pd.DataFrame(columns =['label', 'Beacon_1', 'Beacon_2', 'Beacon_3', 'Beacon_4', 'Beacon_5', 'Beacon_6', 'Beacon_7'])

phone_rssi_list = {'GalaxyA51':[], 'hTCU19e':[], 'hTCU11':[], 'sharp025':[]}
min_rssi = {'GalaxyA51':0, 'hTCU19e':0, 'hTCU11':0, 'sharp025':0}
max_rssi = {'GalaxyA51':0, 'hTCU19e':0, 'hTCU11':0, 'sharp025':0}

for phone in ['GalaxyA51', 'hTCU19e', 'hTCU11', 'sharp025']:
    path = f'./Data/0318_92589_train/{phone}/'
    file_list = [f for f in os.listdir(path) if f[len(f)-3:] == 'csv']

    for rssi_file in file_list:
        df = pd.read_csv(f'{path+rssi_file}', delimiter=',')
        phone_rssi_list[phone] += df['RSSI'].to_list()

    min_rssi[phone] = min(phone_rssi_list[phone])
    max_rssi[phone] = max(phone_rssi_list[phone])
    print(min_rssi[phone],max_rssi[phone])

    for rssi_file in file_list:
        position = table[int(rssi_file[2:4])][int(rssi_file[0:2])]

        print(rssi_file, position)

        df = pd.read_csv(f'{path+rssi_file}', delimiter=',')
        df['UUID'] = df['UUID'].apply(uuid_to_b_num)
        #print(df[0:7])

        rssi = [np.zeros((8))]
        rssi[0][:] = np.nan
        rssi[0][0] = position
        row = 0

        for i in range(len(df) - 1):
            curr = df.iloc[i]['Time']
            next = df.iloc[i+1]['Time']

            rssi_idx = int(df.iloc[i]['UUID'])

            rssi_idx = int(df.iloc[i]['UUID'])
            if rssi_idx > 0 :
                rssi[row][rssi_idx] = (df.iloc[i]['RSSI']-min_rssi[phone])/(max_rssi[phone]-min_rssi[phone])

            if curr != next:  
                row += 1
                rssi.append(np.zeros((8)))
                rssi[row][:] = np.nan
                rssi[row][0] = position

        rssi_idx = int(df.iloc[len(df) - 1]['UUID'])
        #print(rssi_idx)
        if rssi_idx > 0 :
            rssi[row][rssi_idx] = (df.iloc[len(df) - 1]['RSSI']-min_rssi[phone])/(max_rssi[phone]-min_rssi[phone])

        print(rssi)

        new_row = pd.DataFrame(np.array(rssi), columns =['label', 'Beacon_1', 'Beacon_2', 'Beacon_3', 'Beacon_4', 'Beacon_5', 'Beacon_6', 'Beacon_7'])
        wireless_fingerprint = wireless_fingerprint.append(new_row, ignore_index=True)

wireless_fingerprint = wireless_fingerprint.interpolate()
print(wireless_fingerprint)

In [ ]:
# 儲存 min, max rssi 的檔案
import pickle
with open(f'./Data/0318_92589_train/min_rssi.pickle', 'wb') as f:
    pickle.dump(min_rssi, f)
with open(f'./Data/0318_92589_train/max_rssi.pickle', 'wb') as f:
    pickle.dump(max_rssi, f)

In [ ]:
# 儲存 wireless fingerprint training set
wireless_fingerprint.set_index('label').to_csv(f'./Data/0318_92589_train/wireless_training_set.csv')